In [1]:
!pip install gtfs-realtime-bindings
!pip install folium

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for gtfs-realtime-bindings: filename=gtfs_realtime_bindings-1.0.0-py3-none-any.whl size=5995 sha256=451e5f22f88260076793c36f65d82424236c71be1d9e35b6f2a36ecb0f3beff6
  Stored in directory: c:\users\pc\appdata\local\pip\cache\wheels\b6\43\38\17a10a2cdd30cb86acceb42e24e7d2d6bb98b2c59ff8983e20
Successfully built gtfs-realtime-bindings
   ---------------------------------------- 0.0/113.4 kB ? eta -:--:--
   ---------------------------------------- 0.0/113.4 kB ? eta -:--:--
   --- ------------------------------------ 10.2/113.4 kB ? eta -:--:--
   -------------- ------------------------ 41.0/113.4 kB 653.6 kB/s eta 0:00:01
   ---------------------------------------- 113.4/113.4 kB 1.1 MB/s eta 0:00:00


In [5]:
import requests
from google.transit import  gtfs_realtime_pb2
import pandas as pd
import folium as fl

In [6]:
#step 1-Setup the API
API_KEY="TpEpdxa1JRvm7E6jl86mZwGqxlg9oqXj"
API_URL=f"https://otd.delhi.gov.in//api/realtime/VehiclePositions.pb?key=TpEpdxa1JRvm7E6jl86mZwGqxlg9oqXj"

In [8]:
#step-2-Fetch GTFS-realtime data
response=requests.get(API_URL)
if response.status_code !=200:
    raise Exception(f"Error! {response.status_code}:{response.text}")

In [10]:
#step3-Data Parsing to protobuf
feed= gtfs_realtime_pb2.FeedMessage()
feed.ParseFromString(response.content)

224127

In [14]:
#step4- Convert into DataFrame
from datetime import datetime 
data=[]
for entity in feed.entity:
    if entity.HasField('vehicle'):
        vehicle=entity.vehicle
    data.append({
    "VehicleID":vehicle.vehicle.id,
    "TripID":vehicle.trip.trip_id,
    "RouteID":vehicle.trip.route_id,
    "Latitude":vehicle.position.latitude,
    "Longitude":vehicle.position.longitude,
    "Timestamp":datetime.fromtimestamp(vehicle.timestamp) if vehicle.timestamp else None    
    })
    

In [24]:
df=pd.DataFrame(data)
dataset.info()
"Information of captured data"

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2188 entries, 0 to 2187
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   VehicleID  2188 non-null   object        
 1   TripID     2188 non-null   object        
 2   RouteID    2188 non-null   object        
 3   Latitude   2188 non-null   float64       
 4   Longitude  2188 non-null   float64       
 5   Timestamp  2188 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 102.7+ KB


'Information of captured data'

In [28]:
#Now let us see how to make map using folium
#step5- Map visualisation
m=fl.Map(location=[28.65195,77.23149],zoom_start=11) #ye hmne center location bna li hai delhi ki latitude or longitude value le ke
for _, row in df.iterrows():
    fl.Marker(
        location=[row["Latitude"],row["Longitude"]],
        popup=f"Route: {row['RouteID']}<br> Vehicle: {row['VehicleID']}<br>Time:{row['Timestamp']}",
        icon=fl.Icon(color="blue",icon="bus")
).add_to(m)
m.save("Delhi Transport Mapping.html")
print("Map saved soccessfully in the same diractory of code")


Map saved soccessfully in the same diractory of code
